# Module 3 - Activity

During this module you will be exposed to the following topics:

- Understanding of Data (Data Types and Formats)
- Data Collection
- Data Cleaning
- Data Transforms
- Feature Engineering
- Outlier Removal Methodologies
- FEature Ranking and Selection
- Dimensionality Reduction

As you begin to utilize these in your data science process it will be important to understand how to put these into a processing pipeline. With machine learning techniques you will need to repeat the same processing steps for your train data and your test data. A processing pipeline allows for sequential processing of your data where the output of each step becomes the input for the next step. The pipeline should be configurable and extensible. 

You will use the below dataset that is split into train and test and you will implement a data pipeline using at least two processing steps. You may use the built in methods in sklearn for this assignment. 

In [11]:
import pandas as pd
import numpy as np

In [12]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Load the Iris dataset
iris = load_iris()
X = iris.data  # Features
y = iris.target  # Target labels

# Perform an 80-20 train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes of the resulting sets to verify the split
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (120, 4)
X_test shape: (30, 4)
y_train shape: (120,)
y_test shape: (30,)


In [13]:
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

### Preprocessing

Let's place the data in a dataframe for manipulation

In [14]:
iris_df = pd.DataFrame(columns=iris.feature_names,data=iris.data)
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


Are there any duplicates within the frame?

In [15]:
iris_df[iris_df.duplicated() == True]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
142,5.8,2.7,5.1,1.9


It looks like there is one duplicate at index 142. Let's remove it and see what our dataframe looks like

In [16]:
iris_df = iris_df.drop_duplicates()
iris_df[140:145]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
140,6.7,3.1,5.6,2.4
141,6.9,3.1,5.1,2.3
143,6.8,3.2,5.9,2.3
144,6.7,3.3,5.7,2.5
145,6.7,3.0,5.2,2.3


Notice that row 142 has been removed successfully!

Now we can separate per feature

In [17]:
sepal_length = iris_df["sepal length (cm)"]
sepal_width = iris_df["sepal width (cm)"]
petal_length = iris_df["petal length (cm)"]
petal_width = iris_df["petal length (cm)"]

Let's look for some outliers!

Now, we can remove the outliers

Normalization of the data is beneficial to ensure that each of the features are respresentated on the same scale. This can be very nice to achieve for feature ranking/separation.

Let's do some feature ranking! We will use Fisher's Linear Discriminant Ratio

### Data fitting and Transformation

In [18]:
# Fit and transform the training data using the pipeline


In [19]:
# Use the pipeline to transform the test data (X_test)


Please add your Jupyter Notebook in HTML format to the discussion board found under Module 3. You are encouraged to review other students' submissions to check and discuss differences in your approaches. 